# Determine how many nodes connected to annotated nodes

In [1]:
suppressMessages(library(dplyr))

In [2]:
gnps = read.delim('Data/3cda66df1f5b4b738e4d4c673ca6aa1d.tsv')
#gnps[, "cluster.index"] <- as.numeric(gnps[, "cluster.index"])
head(gnps)

cluster.index,number.of.spectra,parent.mass,precursor.charge,precursor.mass,sum.precursor.intensity.,X0,X0.5,X0.76,X1,⋯,LCMS_analysis_plate,race,prescribed_vancomycin,prescribed_magnesium.oxide,Sample_Location_General_Sub2_Text,DBSAA_TP1,prescribed_conjugated.estrogens,prescribed_atenolol,height_m,prescribed_fludrocortisone
1,9,99.512,0,99.512,125140,9,2,0,9,⋯,"1,2,6,3,5,4","Asian,Other,White","0,1","0,1","L_Forehead,L_Palm,R_Nose,L_Nose,L_Backhand","117,114,176,175,174,139,121","0,1","0,1","1.549,1.676,1.854,1.651,1.778,1.727,1.702","0,1"
5,57,108.518,0,108.518,364524,57,8,1,57,⋯,"1,2,6,3,5,4","Asian,Other,White,Black","0,1","0,1","L_Forehead,R_Palm,L_Palm,R_Nose,R_Forehead,R_Backhand,L_Axillary,L_Nose,R_Axillary,L_Backhand","117,114,176,175,174,139,121,125","0,1","0,1","1.549,1.575,1.803,1.676,1.854,1.753,1.6,1.651,1.778,1.727,1.702","0,1"
12,10,113.964,0,113.964,129712,10,1,0,10,⋯,"1,2,6,3,5,4","Asian,Other,White","0,1","0,1","L_Forehead,L_Palm,R_Nose,R_Forehead,R_Axillary,L_Backhand","117,114,175,174,139,121","0,1","0,1","1.549,1.803,1.676,1.753,1.651,1.778","0,1"
14,2,131.975,0,131.975,40576,2,0,0,2,⋯,"1,2,6,4","Other,White","0,1","0,1","R_Forehead,L_Nose",121,"0,1","0,1","1.549,1.753","0,1"
17,4,139.040,0,139.040,291940,4,1,0,4,⋯,"1,2,5,4",White,"0,1","0,1","L_Forehead,L_Nose","176,174,121","0,1","0,1","1.651,1.702","0,1"
18,9,144.983,0,144.983,528576,9,0,0,9,⋯,"1,2,6,3,4","Other,White","0,1","0,1","L_Forehead,R_Nose,R_Backhand,L_Axillary,L_Nose",121,"0,1","0,1","1.549,1.753,1.727","0,1"


In [3]:
length(unique(gnps$LibraryID))

[1] 109

In [4]:
net <- read.delim('Data/828ccf4ce7be41bfb01dae4f54e936d6.pairsinfo', header=TRUE)
dim(net)
head(net)
table <- matrix(rbind(net$CLUSTERID1,net$CLUSTERID2))
table <- unique(table)
dim(table)

[1] 14657     7

CLUSTERID1,CLUSTERID2,DeltaMZ,MEH,Cosine,OtherScore,ComponentIndex
230150,234596,-44.026000,0,0.900879,0.783035,1
227040,236596,-88.063000,0,0.892919,0.688209,1
245387,245612,-11.012000,0,0.900354,0.707705,1
229951,230869,-5.458010,0,0.931996,0.851260,1
223112,231825,-55.033000,0,0.887927,0.695792,1
245204,245207,-0.253052,0,0.909418,0.745468,1


[1] 9644    1

In [5]:
#read data into notebook
annotated <- read.delim("Data/GNPS_Curated_Source_Information_Version2.0 - Curated_Source_Information_Table (18).tsv",  sep="\t", header=TRUE)
annotated <- mmat_sourceinfo[-c(1:3),]
#mmat_sourceinfo <- subset(mmat_sourceinfo, mmat_sourceinfo$Curated_GNPS_Annotation != "disregard")
#mmat_sourceinfo <- subset(mmat_sourceinfo, mmat_sourceinfo$Curated_GNPS_Annotation != "literature limited")
#look at first couple of lines
head(annotated)
#check dimension of data read into notebook
dim(annotated)

ERROR: Error in eval(expr, envir, enclos): object 'mmat_sourceinfo' not found


In [ ]:
unique(annotated$Source)
unique(annotated$Source_Sub1)

In [6]:
# by body location

gnps_with_origin <- data.frame(gnps$cluster.index, gnps$Sample_Location_General_Text, gnps$LibraryID)
gnps_with_origin <- merge(gnps_with_origin, annotated, by.x="gnps.LibraryID", by.y="GNPS_annotation")
#head(gnps_with_origin)
#dim(gnps_with_origin)
#drug_nodes <- gnps_with_origin



#ENTER VALUE HERE....
drug_nodes <- subset(gnps_with_origin, gnps_with_origin$Source == "food")
#drug_nodes <- subset(gnps_with_origin, gnps_with_origin$Source_Sub1 == "food additive")

# give net and drug nodes as input parameters for the function
find_componentIndex <- function(i){
    #net[net[,"CLUSTERID1"]==drug_nodes[i, "gnps.cluster.index"] | net[,"CLUSTERID2"]==drug_nodes[i, "gnps.cluster.index"],]
    #net[net[,"CLUSTERID1"]==drug_nodes[i, "gnps.cluster.index"] | net[,"CLUSTERID2"]==drug_nodes[i, "gnps.cluster.index"],][1, 'ComponentIndex']
    compIndex <- net[net[,"CLUSTERID1"]==drug_nodes[i, "gnps.cluster.index"] | net[,"CLUSTERID2"]==drug_nodes[i, "gnps.cluster.index"],][1, 'ComponentIndex']
    if (compIndex==-1) {
        edges = net[net[,"CLUSTERID1"]==drug_nodes[i, "gnps.cluster.index"],]
        nedges = 0
        return (list(edges=edges, nedges=nedges))
    }
    edges = net[net[, "ComponentIndex"]==compIndex,]
    nedges = nrow(net[net[, "ComponentIndex"]==compIndex,])
    return (list(edges=edges, nedges=nedges))
}

alanslist = lapply(1:nrow(drug_nodes), find_componentIndex)

results <- do.call(rbind, lapply(alanslist, function(x) x$edges))
table_results <- matrix(rbind(results$CLUSTERID1,results$CLUSTERID2))
table_results <- unique(table_results)
print("number of nodes in network")
    dim(table)
print("number of nodes of X origin in network")
    dim(drug_nodes)
    length(drug_nodes[,1])/length(table[,1]) *100                  
print("number of propogated nodes of X origin in network")
    dim(table_results)
    length(table_results[,1])/length(table[,1]) *100
                                 
                                           ####################################                             
# for each group of delta mz find unique files where it occurs
# TIME CONSUMING STEP
drug_nodes_source <- cbind(as.numeric(drug_nodes$gnps.cluster.index), as.character(drug_nodes$gnps.Sample_Location_General_Text))
head(drug_nodes_source)
#dim(drug_nodes_source)

unq <- tapply(drug_nodes_source[,2], drug_nodes_source[,1], function(x) 
                                unique(unlist(sapply(x, function(y) 
                                                         unique(strsplit(y, ',' )[[1]])
                                                     )
                                             )
                                      )
              )

# Some elements had a matrix as answer
# transform all in vectors
unq <- lapply(unq, function(x) as.vector(as.matrix(x)))
#head(unq)
# Transform the list of vectors in a matrix, repeating the 
# label of each list element
mlist <- mapply(function(x,y) cbind(x, y), as.list(names(unq)), unq)
    
    
# concatenate the full matrix
    #head(mlist)
    mtable <- unique(do.call(rbind, mlist))
    length(table(mtable))
    summary(mtable)                       

                
                          ####################################                
                # TIME CONSUMING STEP
gnps_with_origin <- data.frame(gnps$cluster.index, gnps$Sample_Location_General_Text, gnps$LibraryID)
table_results_2 <- as.data.frame(table_results)
table_results_2 <- merge(table_results, gnps_with_origin, by.x="V1", by.y="gnps.cluster.index")
#head(table_results_2)
#dim(table_results_2)

drug_nodes_source <- cbind(as.numeric(table_results_2$V1), as.character(table_results_2$gnps.Sample_Location_General_Text))
#head(drug_nodes_source)
#dim(drug_nodes_source)

unq <- tapply(drug_nodes_source[,2], drug_nodes_source[,1], function(x) 
                                unique(unlist(sapply(x, function(y) 
                                                         unique(strsplit(y, ',' )[[1]])
                                                     )
                                             )
                                      )
              )

# Some elements had a matrix as answer
# transform all in vectors
unq <- lapply(unq, function(x) as.vector(as.matrix(x)))
    #head(unq)
# Transform the list of vectors in a matrix, repeating the 
# label of each list element
mlist <- mapply(function(x,y) cbind(x, y), as.list(names(unq)), unq)

# concatenate the full matrix

    head(mlist)
    #mtable <- unique(do.call(rbind, mlist))
    #length(table(mtable))
    #summary(mtable) 


[1] "number of nodes in network"


[1] 9644    1

[1] "number of nodes of X origin in network"


[1] 32 24

[1] 0.3318125

...

[1] 1.21319

9840,"Hand,Face,Axillary"
43563,"Hand,Face"
72115,"Face,Axillary"
43431,"Hand,Face"
43583,"Hand,Face"
27,Hand


[1] 35

       x             y     
 193169 : 3   Axillary: 8  
 43455  : 3   Face    :26  
 43969  : 3   Hand    :27  
 9840   : 3                
 1261   : 2                
 193173 : 2                
 (Other):45                

x,y
102395,Hand
102395,Face
x,y
1029,Hand
x,y
104132,Hand
104132,Face
104132,Axillary
x,y
11719,Hand
